In [13]:
import numpy as np
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

## Positional encoding

Original positional encoding from "Attention is all you need" paper, merge positional information into the input for parallel computing

In [98]:
d_model = 512
x = torch.rand(1,6,10)
pos = 0
def positional_encoding(tensor):
    batch,length,d_model = tensor.shape
    pe = torch.zeros(length,d_model)
    position = torch.arange(0,length).unsqueeze(1)
    div_term = torch.exp(-math.log(10000.0) * torch.arange(0, d_model, 2).float()/ d_model)
    pe[:,0::2] = torch.sin(position*div_term)
    pe[:,1::2] = torch.cos(position*div_term)
    pe = pe.unsqueeze(0)
    return pe

In [99]:
positional_encoding(x)

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  1.5783e-01,  9.8747e-01,  2.5116e-02,
           9.9968e-01,  3.9811e-03,  9.9999e-01,  6.3096e-04,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  3.1170e-01,  9.5018e-01,  5.0217e-02,
           9.9874e-01,  7.9621e-03,  9.9997e-01,  1.2619e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  4.5775e-01,  8.8908e-01,  7.5285e-02,
           9.9716e-01,  1.1943e-02,  9.9993e-01,  1.8929e-03,  1.0000e+00],
         [-7.5680e-01, -6.5364e-01,  5.9234e-01,  8.0569e-01,  1.0031e-01,
           9.9496e-01,  1.5924e-02,  9.9987e-01,  2.5238e-03,  1.0000e+00],
         [-9.5892e-01,  2.8366e-01,  7.1207e-01,  7.0211e-01,  1.2526e-01,
           9.9212e-01,  1.9904e-02,  9.9980e-01,  3.1548e-03,  1.0000e+00]]])

In [100]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout=0.3, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        #return self.dropout(x)
        return x

In [101]:
a = PositionalEncoding(10)

In [102]:
a.forward(x)

tensor([[[ 0.6268,  1.5623,  0.8246,  1.6400,  0.8334,  1.2400,  0.3692,
           1.4134,  0.0672,  1.5023],
         [ 1.1302,  1.0054,  0.6584,  1.3469,  0.4714,  1.4054,  0.8990,
           1.5506,  0.6671,  1.8118],
         [ 0.9974, -0.0263,  1.1654,  1.8849,  0.9618,  1.0300,  0.0296,
           1.2786,  0.7354,  1.3790],
         [ 0.7998, -0.6143,  1.1517,  1.2161,  0.2881,  1.4628,  0.5263,
           1.3204,  0.8208,  1.7594],
         [ 0.1677, -0.0887,  0.7773,  1.1589,  0.6773,  1.7156,  0.9473,
           1.7304,  0.5003,  1.2489],
         [-0.1962,  1.1408,  0.9834,  1.3166,  0.9852,  1.7799,  0.6950,
           1.2468,  0.2282,  1.2666]]])

## Layer Norm

Layer norm is similar to batch norm but on different dimension - smoother gradients, faster training, and better generalization accuracy. Why we use layer normalization is for dealing with input with different length. 

https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html

In [3]:
x = torch.rand(1,6,10)
x

tensor([[[0.8827, 0.2055, 0.3562, 0.9865, 0.3512, 0.1907, 0.2707, 0.0843,
          0.5042, 0.0954],
         [0.2967, 0.5751, 0.1970, 0.1800, 0.3268, 0.6833, 0.8755, 0.0910,
          0.0446, 0.7569],
         [0.9782, 0.2361, 0.9193, 0.5096, 0.1267, 0.1486, 0.8678, 0.0644,
          0.4013, 0.8395],
         [0.3017, 0.5980, 0.0999, 0.4619, 0.5223, 0.3284, 0.8931, 0.9956,
          0.4821, 0.3691],
         [0.6723, 0.0491, 0.1578, 0.0640, 0.2092, 0.3885, 0.1057, 0.9581,
          0.4982, 0.3878],
         [0.7270, 0.1995, 0.3748, 0.5885, 0.1402, 0.6466, 0.7732, 0.3173,
          0.0909, 0.2985]]])

In [4]:
mean = x.mean(-1, keepdim=True)
std = x.std(-1, keepdim=True)

In [5]:
mean

tensor([[[0.3927],
         [0.4027],
         [0.5091],
         [0.5052],
         [0.3491],
         [0.4157]]])

In [8]:
a = [0.8827, 0.2055, 0.3562, 0.9865, 0.3512, 0.1907, 0.2707, 0.0843,0.5042, 0.0954]
sum(a)/len(a)

0.39274000000000003

In [9]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [10]:
LayerNorm(10)(x)

tensor([[[ 1.5647e+00, -5.9798e-01, -1.1658e-01,  1.8962e+00, -1.3264e-01,
          -6.4535e-01, -3.8982e-01, -9.8512e-01,  3.5612e-01, -9.4952e-01],
         [-3.5738e-01,  5.8122e-01, -6.9343e-01, -7.5084e-01, -2.5580e-01,
           9.4610e-01,  1.5940e+00, -1.0510e+00, -1.2072e+00,  1.1943e+00],
         [ 1.2919e+00, -7.5221e-01,  1.1297e+00,  1.1500e-03, -1.0533e+00,
          -9.9306e-01,  9.8799e-01, -1.2250e+00, -2.9715e-01,  9.0996e-01],
         [-7.5285e-01,  3.4321e-01, -1.4992e+00, -1.6030e-01,  6.3241e-02,
          -6.5392e-01,  1.4349e+00,  1.8140e+00, -8.5523e-02, -5.0353e-01],
         [ 1.0930e+00, -1.0141e+00, -6.4665e-01, -9.6375e-01, -4.7307e-01,
           1.3320e-01, -8.2289e-01,  2.0592e+00,  5.0410e-01,  1.3102e-01],
         [ 1.2469e+00, -8.6579e-01, -1.6347e-01,  6.9238e-01, -1.1031e+00,
           9.2485e-01,  1.4319e+00, -3.9375e-01, -1.3006e+00, -4.6934e-01]]],
       grad_fn=<AddBackward0>)

## Label smoothing

In [ ]:
Lower label 1 to 0.1 prob

## Attention 

$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

In [20]:
x.permute(0, 2, 1)

tensor([[[0.8827, 0.2967, 0.9782, 0.3017, 0.6723, 0.7270],
         [0.2055, 0.5751, 0.2361, 0.5980, 0.0491, 0.1995],
         [0.3562, 0.1970, 0.9193, 0.0999, 0.1578, 0.3748],
         [0.9865, 0.1800, 0.5096, 0.4619, 0.0640, 0.5885],
         [0.3512, 0.3268, 0.1267, 0.5223, 0.2092, 0.1402],
         [0.1907, 0.6833, 0.1486, 0.3284, 0.3885, 0.6466],
         [0.2707, 0.8755, 0.8678, 0.8931, 0.1057, 0.7732],
         [0.0843, 0.0910, 0.0644, 0.9956, 0.9581, 0.3173],
         [0.5042, 0.0446, 0.4013, 0.4821, 0.4982, 0.0909],
         [0.0954, 0.7569, 0.8395, 0.3691, 0.3878, 0.2985]]])

In [17]:
x.transpose(-2, -1)

tensor([[[0.8827, 0.2967, 0.9782, 0.3017, 0.6723, 0.7270],
         [0.2055, 0.5751, 0.2361, 0.5980, 0.0491, 0.1995],
         [0.3562, 0.1970, 0.9193, 0.0999, 0.1578, 0.3748],
         [0.9865, 0.1800, 0.5096, 0.4619, 0.0640, 0.5885],
         [0.3512, 0.3268, 0.1267, 0.5223, 0.2092, 0.1402],
         [0.1907, 0.6833, 0.1486, 0.3284, 0.3885, 0.6466],
         [0.2707, 0.8755, 0.8678, 0.8931, 0.1057, 0.7732],
         [0.0843, 0.0910, 0.0644, 0.9956, 0.9581, 0.3173],
         [0.5042, 0.0446, 0.4013, 0.4821, 0.4982, 0.0909],
         [0.0954, 0.7569, 0.8395, 0.3691, 0.3878, 0.2985]]])

In [14]:
def attention(query, key, value, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    p_attn = F.softmax(scores, dim = -1) ##changed dim to 1
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [15]:
attention(x,x,x)

(tensor([[[0.6724, 0.3023, 0.3786, 0.5038, 0.2763, 0.3762, 0.6315, 0.3886,
           0.3473, 0.4496],
          [0.6270, 0.3331, 0.3621, 0.4557, 0.2805, 0.4079, 0.6778, 0.3986,
           0.3192, 0.4877],
          [0.6765, 0.3087, 0.4109, 0.4842, 0.2651, 0.3767, 0.6682, 0.3660,
           0.3358, 0.4929],
          [0.6179, 0.3332, 0.3389, 0.4589, 0.2940, 0.3952, 0.6602, 0.4565,
           0.3428, 0.4613],
          [0.6411, 0.3063, 0.3453, 0.4530, 0.2813, 0.3902, 0.6229, 0.4564,
           0.3501, 0.4549],
          [0.6510, 0.3151, 0.3717, 0.4770, 0.2755, 0.3948, 0.6570, 0.3982,
           0.3311, 0.4687]]]),
 tensor([[[0.2003, 0.1365, 0.1949, 0.1608, 0.1389, 0.1686],
          [0.1351, 0.1975, 0.1882, 0.1791, 0.1304, 0.1697],
          [0.1572, 0.1534, 0.2479, 0.1515, 0.1249, 0.1652],
          [0.1412, 0.1589, 0.1649, 0.2254, 0.1512, 0.1584],
          [0.1512, 0.1433, 0.1685, 0.1873, 0.1909, 0.1588],
          [0.1586, 0.1613, 0.1927, 0.1697, 0.1372, 0.1805]]]))